In [1]:
import pandas as pd
import sys
from sklearn.pipeline import Pipeline
import numpy as np
import string
import os
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

import xgboost

import tsfresh
from tsfresh import extract_features


In [2]:
os.path.abspath('..')
sys.path.append(os.path.abspath('..'))


In [3]:
import importlib


import board_helpers
from board_helpers.move_info import MoveInfo
from board_helpers.tiles_counter import TilesCoutner
from board_helpers.board_consts import TileType
importlib.reload(board_helpers)
importlib.reload(board_helpers.move_info)
importlib.reload(board_helpers.tiles_counter)
importlib.reload(board_helpers.board_consts)



import transformers
from transformers.columns_setter_transformer import ColumnsSetterTransformer
from transformers.extract_set_column_transformer import ExtractSetColumnsTransformer
from transformers.name_dropper_transformer import NameDropperTransformer
from transformers.select_transformer import SelectTransformer
from transformers.select_rows_transformer import SelectRowsTransformer
from transformers.series_from_group_transformer import SeriesFromGroupTransformer
from transformers.map_set_transformer import MapSetTransformer
from transformers.one_hot_encoder_transformer import OneHotEncoderTransformer
from transformers.add_to_dict_transformer import AddToDictTransformer
from transformers.get_from_dict_transformer import GetFromDictTransformer
from transformers.select_pos_rows_transformer import SelectPosRowsTransformer
importlib.reload(transformers)
importlib.reload(transformers.columns_setter_transformer)
importlib.reload(transformers.extract_set_column_transformer)
importlib.reload(transformers.name_dropper_transformer)
importlib.reload(transformers.select_transformer)
importlib.reload(transformers.select_rows_transformer)
importlib.reload(transformers.series_from_group_transformer)
importlib.reload(transformers.map_set_transformer)
importlib.reload(transformers.one_hot_encoder_transformer)
importlib.reload(transformers.add_to_dict_transformer)
importlib.reload(transformers.get_from_dict_transformer)
importlib.reload(transformers.select_pos_rows_transformer)

import functions
from functions.bot_extractor import BotExtarctor
from functions.is_bot_extractor import IsBotExtarctor
importlib.reload(functions)
importlib.reload(functions.is_bot_extractor)
importlib.reload(functions.bot_extractor)

import processors
from processors.basic_pre_processor import BasicPreProcessor
importlib.reload(processors)
importlib.reload(processors.basic_pre_processor)

import builders
from builders.preprocessor_builder import PreprocessorBuilder
from builders.reg_pipe_builder import RegPipeBuilder
importlib.reload(builders)
importlib.reload(builders.preprocessor_builder)
importlib.reload(builders.reg_pipe_builder)

import searcher
import searcher_results_orgenizer
from searcher import Searcher
from searcher_results_orgenizer import SearcherResultsOrgenizer
importlib.reload(searcher)
importlib.reload(searcher_results_orgenizer)







<module 'searcher_results_orgenizer' from 'c:\\Users\\Bina4\\Desktop\\Guy_hafifa\\scrabble\\searcher_results_orgenizer.py'>

In [4]:
games = pd.read_csv('../data/games.csv', index_col='game_id')
train = pd.read_csv('../data/train.csv', index_col='game_id')
turns = pd.read_csv('../data/turns.csv', index_col='game_id')

In [5]:
m = MoveInfo('B10', None)
print(m.x, m.y, m.isVertical)

2 10 True


In [6]:
TilesCoutner('F7', "hh")()

{<TileType.N: 1>: 2,
 <TileType.L2: 2>: 0,
 <TileType.L3: 3>: 0,
 <TileType.W2: 4>: 0,
 <TileType.W3: 5>: 0}

In [7]:
type(1)

int

In [8]:
TileType(1)

<TileType.N: 1>

In [9]:
# s = turns.apply(lambda x: TilesCoutner(x['location'], x['move'])()[TileType.N] if type(x['location'])==str and type(x['move']) == str else 0, axis=1)

In [10]:
# s

In [11]:
turns

,turn_number,nickname,rack,location,move,points,score,turn_type
game_id,,,,,,,,
1,1,BetterBot,DDEGITT,8G,DIG,10,10,Play
1,2,stevy,AEHOPUX,7H,HAP,18,18,Play
1,3,BetterBot,DEELTTU,6I,LUTE,16,26,Play
1,4,stevy,EMORSUX,5K,UM,16,34,Play
1,5,BetterBot,ACDEITU,L5,..DICATE,28,54,Play
...,...,...,...,...,...,...,...,...
72773,22,adola,ABINRRU,15N,IN,18,376,Play
72773,23,HastyBot,EGHIIP,H12,.HIG,24,331,Play
72773,24,adola,ABRRU,15E,BRA.,7,383,Play


In [12]:
G_NAME = 'games'
T_NAME = 'turns'
DATA_NAME = 'train'

names = ['BetterBot', 'STEEBot', 'HastyBot']

prePipe = Pipeline([('get_relavent_turns', SelectRowsTransformer(train.index, target=T_NAME)),
                     ('get_relavent_games',  SelectRowsTransformer(train.index.unique(), target=G_NAME)),
                     ('train_set_is_player', ExtractSetColumnsTransformer({'is_player': IsBotExtarctor(names, 'nickname', True)},
                                                                           src=DATA_NAME, dest=DATA_NAME)),
                    ('turns_set_is_player', ExtractSetColumnsTransformer({'is_player': IsBotExtarctor(names, 'nickname', True)},
                                                                           src=T_NAME, dest=T_NAME)),  
                     ('get_bot_rating', ExtractSetColumnsTransformer({'bot_rating': lambda train: train[~train['is_player']]['rating']},
                                                                      src=DATA_NAME, dest=G_NAME)),
                     ('get_bots_names', ExtractSetColumnsTransformer({'bot_name': lambda train: train[~train['is_player']]['nickname']},
                                                                     src=DATA_NAME, dest=G_NAME)),
                    ('data_drop_bot_rating', SelectPosRowsTransformer('is_player', target=DATA_NAME)),
                    ])

In [13]:
preprocessor = PreprocessorBuilder(games, G_NAME, turns, T_NAME, prePipe).build()
n_games, n_turns, n_ratings = preprocessor.process(train, DATA_NAME)

In [14]:
n_turns = n_turns.reset_index()

In [15]:
n_turns

,game_id,turn_number,nickname,rack,location,move,points,score,turn_type,is_player
0,1,1,BetterBot,DDEGITT,8G,DIG,10,10,Play,False
1,1,2,stevy,AEHOPUX,7H,HAP,18,18,Play,True
2,1,3,BetterBot,DEELTTU,6I,LUTE,16,26,Play,False
3,1,4,stevy,EMORSUX,5K,UM,16,34,Play,True
4,1,5,BetterBot,ACDEITU,L5,..DICATE,28,54,Play,False
...,...,...,...,...,...,...,...,...,...,...
2782573,72773,22,adola,ABINRRU,15N,IN,18,376,Play,True
2782574,72773,23,HastyBot,EGHIIP,H12,.HIG,24,331,Play,False
2782575,72773,24,adola,ABRRU,15E,BRA.,7,383,Play,True
2782576,72773,25,HastyBot,EIP,7A,PIE,11,342,Play,False


In [17]:
letters = list(string.ascii_letters)
letters.append('.')

turn_pipe = Pipeline([
                      ('hot', OneHotEncoderTransformer({'turn_type': n_turns['turn_type'].unique()})),
                      # ('count_letters_used',
                      #   MapSetTransformer({letter: (lambda x, letter=letter: x.count(letter) if type(x) == str else 0, 'move') for letter in letters})),
                      # ('count_tiles_used', 
                      #   ExtractSetColumnsTransformer(
                      #      {str(t_type): (lambda turns, t_type=t_type: turns.apply(lambda x: TilesCoutner(x['location'], x['move'])()[t_type] if type(x['location'])==str and type(x['move'])==str else 0, axis=1)) for t_type in TileType}
                      #   )),
                      ('bool_to_int', ExtractSetColumnsTransformer({'is_player': lambda turns: turns['is_player'].astype(int)})),
                      ('drops', NameDropperTransformer(['turn_type', 'nickname', 'move', 'location', 'rack'])),
                    ])
t_turns = turn_pipe.transform(n_turns)

In [18]:
t_turns

,game_id,turn_number,points,score,is_player,turn_type_Challenge,turn_type_End,turn_type_Exchange,turn_type_Pass,turn_type_Play,turn_type_Six-Zero Rule,turn_type_Timeout,turn_type_nan
0,1,1,10,10,0,0,0,0,0,1,0,0,0
1,1,2,18,18,1,0,0,0,0,1,0,0,0
2,1,3,16,26,0,0,0,0,0,1,0,0,0
3,1,4,16,34,1,0,0,0,0,1,0,0,0
4,1,5,28,54,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2782573,72773,22,18,376,1,0,0,0,0,1,0,0,0
2782574,72773,23,24,331,0,0,0,0,0,1,0,0,0
2782575,72773,24,7,383,1,0,0,0,0,1,0,0,0
2782576,72773,25,11,342,0,0,0,0,0,1,0,0,0


In [19]:
extracted_features = extract_features(t_turns, column_id="game_id", column_sort="turn_number", n_jobs=0)

Feature Extraction:   0%|          | 0/20 [00:00<?, ?it/s]